# Land Use Categorization for TMD23
This is intended to be a demo implementing the new definitions of land use categorizations for the Trip Generation step of the TDM23 (travel demand model in development). This notebook starts with categorization of Central Business Districts (CBDs).


Started March 30th, 2021 - Margaret Atkinson

#### Requirements:
This is running as a ipynb in Jupyter Notebooks in an Anaconda environment that contains the following:
- Python 3.7
- numpy
- pandas
- openmatrix
- matplotlib
- descartes
- ipympl
- geopandas
- nb_conda
- nb_conda_kernels
- folium
- branca
- jenkspy (INSTALL FIRST)

For more information on how this environment was set up, see: https://github.com/bkrepp-ctps/mde-prototype-python

#### Just a note about environments:

This script seems to require pyproj > 2.2. The environment base_py_37_branca seems to have this.

Hypothesis: In order for pyproj to deal with CRS (coordinate reference systems) in the geodataframes - we need to be using the most recent version of pyproj. See: https://geopandas.org/docs/user_guide/projections.html#upgrading-to-geopandas-0-7-with-pyproj-2-2-and-proj-6

This changes how we specify CRSs. Also note that going from pivot table back to geodataframe (even though the pivot table has geometry) we need to specify the CRS - which should be the same CRS that the data came out of.

Using a version of pyproj that uses the {'init': 'ESPG4326'} format is giving me inconsistent results - sometimes maintaining the CRS and sometimes not. This causes the buffers to be larger than desired and causes problems for spatial join.


#### Links about Decay Functions
- https://aceso.readthedocs.io/_/downloads/en/latest/pdf/
- https://geographicdata.science/book/notebooks/04_spatial_weights.html
- https://github.com/pysal/pysal


## Definitions:
Higher classes are exclusions for lower classes (e.g. if TAZ is both CBD and Dense Urban, it will only be CBD).
### CBD
CBD: A TAZ is marked as a CBD if it has Urban density AND if it is within 0.5 miles of two heavy rail (subway) stops from two different lines or within  0.25 miles of a green line stop that serves all four branches.

Methodology Plan: Buffer each heavy rail stop and dissolve by line to get one buffer per heavy rail line and one buffer for the green line (light rail) for the four stops that serve all four branches of the green line.. 
 - Method 2: Intersect heavy rail buffers (and 4 light rail buffers) to get polygons where buffers overlap each other. Dissolve those polygons into one polygon. If dissolved buffer polygon covers over 50% of the TAZ and the TAZ has Urban Density, it is a CBD.


Data: (shapefiles)
- TAZ's (polygons)
- Heavy Rail Stops with Line field (points)
- Light Rail Stops with Line field (points)

Notes:
- Additional filter: 
    - The TAZ must be over 50% covered by the overlap of two heavy rail stop buffers of different lines (like red/orange)

### Dense Urban
#### Basic Definition:

OLD:
(0.5 mile buffer for heavy rail stops) OR (0.5 mile buffer for CR stops AND Urban density of landuse) OR (0.5 mile buffer for 5 mins headway overlapping bus routes AND Urban density of model landuse)

- *headway is frequency - buses must run every 5 minutes in the AM model period*
- we will be using the combined headway procedure from the model for nodes where the headway 5 min or less means that there are 36 buses or more per 3 hours (AM time period). These nodes will be buffered to meet the criteria for overlapping bus routes (but the buffers need not overlap).

NEW: 
(0.5 mile buffer for heavy rail stops) OR (0.5 mile buffer for CR stops AND Urban density of landuse) OR (0.5 mile buffer for all nodes that have a frequency >= 36 during the AM Peak Period AND Urban density of model landuse)

#### GIS Definition:

A TAZ is defined as Dense Urban if the TAZ does not meet the criteria for a CBD and meets one of the follow criteria:
1. TAZ must intersect with a 0.5 mile buffer for heavy rail stops OR 
2. (a TAZ must intersect with a 0.5 mile buffer for CR stops AND meet the landuse density threshold for Urban density) OR 
3. (a TAZ must intersect with a 0.5 mile buffer for a node that has a frequency >= 36 during the AM Peak Period in the model AND the TAZ must meet the landuse density threshold for Urban density)

#### Density (calculated at the TAZ level):

Density = (Population + Employment)/Area in Sq Mi
- [1] Urban density range is 10,000 - 3,805,119 ;  
- [2] Suburban density range is 5,000 - 9,999 ;  
- [3] Rural density is 0 - 4,999 ; 
- Boston Core includes TAZs 1-155 (we don't need this usually urban density, will fall in Urban density)

#### Notes: 
- Excludes CBD
- No overlapping heavy rail stops
- the 5 min node frequency file is MBTA only, the 15 min node frequency file is all bus routes in the state (private buses, RTAs, etc.).

#### Data:
- Heavy rail stops (points) = Blue, Orange, Red lines (including Mattapan which counts as Red)
- Commute Rail stops (points)
- TAZs with population and employment (polygons)
- Bus routes with AM headway (5 minutes) 
    - (lines) = Modes 1, 2, 3, use AM_Headway <= 5
    - only MBTA


### Fringe Urban
(1  mile buffer for heavy rail stops) OR (0.5 mile buffer for light rail stops) OR (0.5 mile buffer for all nodes that have a frequency >= 36 during the AM Peak Period in the model AND Suburban density) OR (0.5 mile buffer for CR stops AND Suburban density of model  landuse)

***Green line and Silver line are light rail (multiple modes, use them all) 


#### GIS Definition:

A TAZ is defined as Fringe Urban if the TAZ does not meet the criteria for a CBD or Dense Urban and meets one of the follow criteria:
1. TAZ must intersect with a 1 mile buffer for heavy rail stops OR 
2. (TAZ must intersect with a 0.5 mile buffer for light rail stops) OR
3. (a TAZ must intersect with a TAZ must intersect with a 0.5 mile buffer for a node that has frequency >= 12 during the AM Peak Period in the model AND the TAZ must meet the landuse density threshold for suburban density) OR
4. (a TAZ must intersect with 0.5 mile buffer for CR stops AND meet the landuse density threshold for suburban density)

#### Data:
- Heavy rail stops (points) = Blue, Orange, Red lines (including Mattapan which counts as Red)
- Commuter Rail stops (points)
- Light Rail stops (points) = Green and Silver lines.
- TAZs with population and employment (polygons)
- Bus routes with AM headway (15 minutes) 
    - NOT only MBTA

### Suburban
Basic Definition:
0.5 mile buffer for any transit line (exclude CR , private buses)

GIS Definition:
TAZs that are not CBD, Dense Urban, or Fringe Urban AND intersect with a 0.5 mile buffer for any transit line (excluding CR and private bus lines).

### Rural
Definition of exclusion:
All TAZs that are not CBD, Dense Urban, Fringe Urban, or Suburban.

In [ ]:
#IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import copy
import folium

In [ ]:
#Import Data
#import TAZ shapefile

taz = "G:\Regional_Modeling\Projects\Landuse Type_Project\To Margrate\TAZ_CRS.shp"
    #load into a geodataframe
taz = gpd.read_file(taz)
    #filter to just be TAZ ID and geometry
taz = taz[['ID',"TOT_EMP", "TOT_POP",'geometry', 'AREA']]
    #rename ID to TAZ_ID
taz = taz.rename(columns={"ID":"TAZ_ID"})

#import all transit routes
routes = r"G:\Regional_Modeling\Projects\Landuse Type_Project\To Margrate\LRTP Routes_05.11.2021\Model_Routes_CRS.shp"
routes = gpd.read_file(routes)
routes = routes.query('SCEN_00 == 1') #filter to include just in current scenario

#import all stops (must have line column)
stops = r"G:\Regional_Modeling\Projects\Landuse Type_Project\To Margrate\LRTP Routes_05.11.2021\Model_Stops_CRS.shp"
stops = gpd.read_file(stops)
stops = stops[stops['ROUTE_ID'].isin(routes['ROUTE_ID'])] #filter to include just in current scenario (e.g. in routes)

#transform TAZ and stop layers' CRS to CTPS Standard - "EPSG:26986" (Massachusetts State Plane)
taz = taz.to_crs("EPSG:26986")
stops = stops.to_crs("EPSG:26986")
routes = routes.to_crs("EPSG:26986")

#import 5min headway and 15min headway
f5 = r'G:\Regional_Modeling\Projects\Landuse Type_Project\Node GISDK work\results\Aggr_Stops_5minheadway.csv'
f15 = r"G:\Regional_Modeling\Projects\Landuse Type_Project\Node GISDK work\results\Aggr_Stops_15minheadway.csv"
f5 = pd.read_csv(f5, header = 0, names = ['NODE', 'FREQ5']) #data does not come with header for either csv
f15 = pd.read_csv(f15, header = 0, names = ['NODE', 'FREQ15'])


## Filtering and Cleaning the Data
This section is about filtering and setting up data products for use in all future sections. This means that the analysis for each section doesn't involve setting up data and data that is shared is stored in a common place.

In [ ]:
#FILTER THE STOPS

#HEAVY RAIL (CBD, DENSE URBAN, FRINGE URBAN)
#filter all stops to just heavy rail stops (and make a copy)
hr_stops = copy.deepcopy(stops.loc[stops['MODE'].isin([5,6,7,8])]) 
#mode 4 is Green, 5 is Red, Mattapan is 6 (Ashmont to Mattapan), Blue is 8, Orange is 7
    #Treat Mattapan as 5 (red)

hr_stops.loc[hr_stops['MODE'] == 6, 'MODE'] = 5 #this line does the same thing as above
#filter
hr_stops = hr_stops.loc[:,['ID', 'MODE', 'STOP_NAME', 'geometry']]
#check
hr_stops[hr_stops['MODE'].isin([6])] 


In [ ]:
#COMMUTER RAIL (DENSE URBAN, FRINGE URBAN)
#filter all stops to just commuter rail stops (and make a copy)
cr_stops = copy.deepcopy(stops.loc[stops['MODE'].isin([9,32,33,34,35,36,37,38,39,40])]) 
#filter
cr_stops = cr_stops.loc[:,['ID', 'MODE', 'STOP_NAME', 'geometry']]
cr_stops.head()

In [ ]:
#LIGHT RAIL (FRINGE URBAN & CBD)
#filter all stops to just light rail stops (and make a copy)
lr_stops = copy.deepcopy(stops.loc[stops['MODE'].isin([4,12,13])]) 
#filter
lr_stops = lr_stops.loc[:,['ID', 'MODE', 'STOP_NAME', 'geometry']]
all4_lrStop = copy.deepcopy(lr_stops.loc[lr_stops['STOP_NAME'].isin(['PARK STREET GREEN', 'BOYLSTON','ARLINGTON','COPLEY SQUARE'])]) 
all4_lrStop = all4_lrStop.loc[:,['ID', 'MODE', 'STOP_NAME', 'geometry']]
all4_lrStop

In [ ]:
#Just get Nodes from Stops (DENSE URBAN, FRINGE URBAN)
nodes = copy.deepcopy(stops.groupby('NODE').first().reset_index()) #this makes it one record per node id
nodes = nodes[['NODE', 'geometry']]
    #merge with the frequency data
nodes = nodes.merge(f5, how = 'left', on = 'NODE')
nodes = nodes.merge(f15, how = 'left', on = 'NODE')
nodes = nodes.dropna(how = 'any', thresh=3) #make sure if both are null not using.
nodesf5 = nodes.query('FREQ5 >= 36') #this filters so headway < 5min and only bus modes 1,2,3 (because thats whats in the file)
nodesf15 = nodes.query('FREQ15 >= 12') #this filters so headway < 15min
nodesf15

In [ ]:
#ROUTES FOR SUBURBAN
sub_routes = copy.deepcopy(routes.loc[routes['MODE'].isin([1,2,3,4,5,6,7,8,12,13,17,18,19,20,21,22,41,42,43])]) 
sub_routes

In [ ]:
#Calculate Density for TAZs
#Density = (Population + Employment)/Area in Sq Mi

taz['DensPy'] = (taz['TOT_POP']+taz['TOT_EMP'])/taz['AREA']

#Flag Urban, Suburban, Rural
taz['Den_Flag'] = np.where(taz['DensPy'] >= 10000, 1, 
                          np.where(taz['DensPy'] < 5000, 3, 2))
taz.loc[taz['Den_Flag']<3]


## CBD

In [ ]:
#BUFFER CELL

#convert the 0.5 mile buffer distance into meters (CRS is in meters)
buf5 = 0.5*1609.34

#make a copy of hr_stops to buffer
hr_buf = copy.deepcopy(hr_stops) #new used to just be hr_buf = hr_stops

#Buffer the heavy rail stops
    #keep attribute data by replacing the geometry column
hr_buf['geometry']= hr_buf.buffer(buf5)

#group by line and dissolve
hr_buf_dis = hr_buf.dissolve(by='MODE').reset_index()

#only get what we need
hr_buf_dis = hr_buf_dis[['MODE', 'geometry']]
hr_buf_dis.plot()

In [ ]:
#BUFFER CELL

#convert the 0.25 mile buffer distance into meters (CRS is in meters)
buf25 = 0.25*1609.34

#make a copy of lr_stops to buffer
lr4_buf = copy.deepcopy(all4_lrStop) #new used to just be hr_buf = hr_stops

#Buffer the light rail stops
lr4_buf['geometry']= lr4_buf.buffer(buf25)

#group and dissolve
lr4_buf_dis = lr4_buf.dissolve(by='MODE').reset_index()

#only get what we need
lr4_buf_dis = lr4_buf_dis[['MODE', 'geometry']].reset_index()
lr4_buf_dis

### CBD: Intersect and Count Method TWO!

In [ ]:
#INTERSECT TRY 2
#split into individual geodataframes to intersect
red = hr_buf_dis[hr_buf_dis['MODE'] == 5]
blue = hr_buf_dis[hr_buf_dis['MODE'] == 7]
orange = hr_buf_dis[hr_buf_dis['MODE'] == 8]

#get all overlaps between route buffers
rb = gpd.overlay(red, blue, how='intersection')
ob = gpd.overlay(orange, blue, how='intersection')
ro = gpd.overlay(red, orange, how= 'intersection')

#put overlaps into one file with overlaps not overlapping (union)
rbob = gpd.overlay(rb, ob, how='union')
rbobro = gpd.overlay(rbob, ro, how='union',keep_geom_type=False)

#add in the light rail
rbobro = rbobro.append(lr4_buf)
#turn ino one row multi part polygon
rbobro['MODE'] = '578+g4'
rbobro = rbobro.dissolve(by='MODE').reset_index()
#get only fields we need
rbobro = rbobro[['MODE', 'geometry']]
rbobro

taz_hr = gpd.overlay(taz, rbobro, how='intersection')
taz_hr
taz_hr.plot()

In [ ]:
#COUNT TRY 2
#caculate area
#taz_hr.area
taz_hr['area_int'] = taz_hr.area
taz['taz_area'] = taz.area
#join intersecting areas back to main TAZ
taz_int = taz.merge(taz_hr, how='left', on='TAZ_ID')
#get percent of taz that is intersecting
taz_int['perc_hr'] = taz_int['area_int']/taz_int['taz_area']

#flag TAZ's
#flag as cbd if taz is over half covered by buffer, density is urban 
taz_int['CBD_Flag'] = np.where(((taz_int.perc_hr >0.5)&(taz_int.Den_Flag_x ==1)), 1, 0)


#turn back into geodataframe - geometry is TAZ (not intersections of TAZ and buffers)
taz_int = taz_int.rename(columns={'geometry_x':'geometry', 'TOT_EMP_x' : 'TOT_EMP', 'TOT_POP_x': 'TOT_POP',
                                  'DensPy_x':'DensPy', 'Den_Flag_x': 'Den_Flag'})
taz_int = gpd.GeoDataFrame(taz_int)
taz_int = taz_int[['TAZ_ID', 'TOT_EMP', 'TOT_POP', 'geometry', 'DensPy', 'Den_Flag', 'CBD_Flag']]#, 'CBD_Perc']]

#make sure that taz_int doesn't have duplicates because of light rail and heavy rail being dif buf polygons
#first sort so that when deleting duplicates, deleting the ones that didn't pass CBD muster
taz_int = taz_int.sort_values(by = ['CBD_Flag'], ascending = False)
#delete duplicates
taz_int = taz_int.drop_duplicates('TAZ_ID')

#make testing outputs
CBD_TAZ2 = taz_int[taz_int['CBD_Flag'] == 1]
#CBD_TAZperc = taz_int[taz_int['CBD_Perc'] == 1]

CBD_TAZ2 = CBD_TAZ2[['TAZ_ID', 'geometry', 'CBD_Flag']]
CBD_TAZ2.plot()
taz_int.head()

## Dense Urban 

In [ ]:
#Buffering everything for Dense Urban
#convert the 0.5 mile buffer distance into meters (CRS is in meters)
buf5 = 0.5*1609.34

#make copies of everything to buffer
cr_buf = copy.deepcopy(cr_stops) #new used to just be hr_buf = hr_stops
hr_buf = copy.deepcopy(hr_stops) #replaces previous hr_buf, but idential to previous hr_buf
nodesf5_buf = copy.deepcopy(nodesf5)

#Buffer everything!!!
cr_buf['geometry']= cr_buf.buffer(buf5)
hr_buf['geometry']= hr_buf.buffer(buf5)
nodesf5_buf['geometry'] = nodesf5_buf.buffer(buf5)

#dissolve hr and cr and bus stops with headway < 5 min
hr_buf_dis_du = hr_buf.dissolve().reset_index()
cr_buf_dis = cr_buf.dissolve().reset_index()
nodesf5_buf_dis = nodesf5_buf.dissolve().reset_index()


In [ ]:
#Calculate Flag Field
taz_du = copy.deepcopy(taz_int) #join basic taz with taz_int

#(already did bus above)
cr_list = []
hr_list = []
f5_list = []
count = 0

for index, row in gpd.overlay(hr_buf_dis_du, taz_du, how = 'intersection').iterrows():
    hr_list.append(row['TAZ_ID'])
for index, row in gpd.overlay(cr_buf_dis, taz_du, how = 'intersection').iterrows():
    cr_list.append(row['TAZ_ID'])
for index, row in gpd.overlay(nodesf5_buf_dis, taz_du, how = 'intersection').iterrows():
    f5_list.append(row['TAZ_ID'])


taz_du['DU_Flag'] = np.where(((taz_du['TAZ_ID'].isin(hr_list))) | ((taz_du['TAZ_ID'].isin(cr_list)) & (taz_du['Den_Flag'] == 1)) | ((taz_du['TAZ_ID'].isin(f5_list)) & (taz_du['Den_Flag'] == 1)), 1,0)
taz_du.query('DU_Flag == 1') #862 records without CBD, 941 with CBD

In [ ]:
taz_du.query('DU_Flag == 1').plot()

## Fringe Urban

In [ ]:
#Buffering everything for Fringe Urban
#convert the 0.5 and 1 mile buffer distances into meters (CRS is in meters)
buf5 = 0.5*1609.34
buf1 = 1*1609.34 #obvious but for clarity

#make copies of everything to buffer
cr_buf = copy.deepcopy(cr_stops) 
hr_buf1 = copy.deepcopy(hr_stops) 
nodesf15_buf = copy.deepcopy(nodesf15)
lr_buf = copy.deepcopy(lr_stops)

#Buffer everything!!!
cr_buf['geometry']= cr_buf.buffer(buf5)
hr_buf1['geometry']= hr_buf1.buffer(buf1) #buffer is now 1 mi
nodesf15_buf['geometry']= nodesf15_buf.buffer(buf5)
lr_buf['geometry']= lr_buf.buffer(buf5)

#dissolve hr and cr
hr_buf_dis_fu = hr_buf1.dissolve().reset_index()
cr_buf_dis = cr_buf.dissolve().reset_index()
nodesf15_buf_dis = nodesf15_buf.dissolve().reset_index()
lr_buf_dis = lr_buf.dissolve().reset_index()

lr_buf_dis.plot()

In [ ]:
#Calculate Flag Field
taz_fu = copy.deepcopy(taz_du) #join basic taz with taz_int

cr_list = []
hr_list = [] #1 mile now
f15_list = []
lr_list = []

#For each row in the intersection output - take all the TAZs (will only be ones that intersect) and put in a list
for index, row in gpd.overlay(hr_buf_dis_fu, taz_fu, how = 'intersection').iterrows():
    hr_list.append(row['TAZ_ID'])
for index, row in gpd.overlay(cr_buf_dis, taz_fu, how = 'intersection').iterrows():
    cr_list.append(row['TAZ_ID'])
for index, row in gpd.overlay(nodesf15_buf_dis, taz_fu, how = 'intersection').iterrows():
    f15_list.append(row['TAZ_ID'])
for index, row in gpd.overlay(lr_buf_dis, taz_fu, how = 'intersection').iterrows():
    lr_list.append(row['TAZ_ID'])

#flag if fits any of the conditions
taz_fu['FU_Flag'] = np.where((taz_fu['TAZ_ID'].isin(hr_list)) | 
                             (taz_fu['TAZ_ID'].isin(lr_list)) |
                             ((taz_fu['TAZ_ID'].isin(cr_list)) & (taz_fu['Den_Flag'] == 2)) | 
                             ((taz_fu['TAZ_ID'].isin(f15_list)) & (taz_fu['Den_Flag'] == 2)), 1,0)
taz_fu.query('FU_Flag == 1') #should be 482 (no CBD or DU), 1188 with CBD and DU

In [ ]:
taz_fu.query('FU_Flag == 1').plot()

## Suburban

In [ ]:
#do the buffers first
#convert miles to meters for CRS
buf5 = 0.5*1609.34
#make copy to copy buffer geo into
sub_buf = copy.deepcopy(sub_routes)
#do the buffers
sub_buf['geometry']= sub_buf.buffer(buf5)
#dissolve the buffers
sub_buf_dis = sub_buf.dissolve().reset_index()
sub_buf_dis.plot()

In [ ]:
#calculate flag field
taz_su = copy.deepcopy(taz_fu)
#make list of all taz that intersect with sub_buf_dis
sub_list = []
for index, row in gpd.overlay(sub_buf_dis, taz_su, how = 'intersection').iterrows():
    sub_list.append(row['TAZ_ID'])
    
#flag everything that intersects with one of the buffers (included in list)
taz_su['SUB_Flag'] = np.where(taz_su['TAZ_ID'].isin(sub_list), 1, 0)
taz_su.query("SUB_Flag == 1").plot()

## Rural

In [ ]:
taz_ru = copy.deepcopy(taz_su)
taz_ru['R_Flag'] = np.where((taz_ru['CBD_Flag'] == 0) & (taz_ru['DU_Flag'] == 0) & (taz_ru['FU_Flag'] == 0) & (taz_ru['SUB_Flag'] == 0), 1, 0)
taz_ru.query('R_Flag == 1').plot()
taz_ru

## Make Land Use Type Column


In [ ]:
taz_ru['LU_Type'] = np.where(taz_ru['CBD_Flag'] == 1, 1, 
                            np.where((taz_ru['CBD_Flag'] != 1) & (taz_ru['DU_Flag'] == 1), 2,
                            np.where((taz_ru['CBD_Flag'] != 1) & (taz_ru['DU_Flag'] != 1) & (taz_ru['FU_Flag'] == 1), 3,
                            np.where((taz_ru['CBD_Flag'] != 1) & (taz_ru['DU_Flag'] != 1) & (taz_ru['FU_Flag'] != 1) & (taz_ru['SUB_Flag'] == 1), 4, 5 
                            ))))
#CBD = 1, DU = 2, FU = 3, SUB = 4, Rural = 5
taz_ru

In [ ]:
m = folium.Map()
folium.Choropleth(taz_ru, data=taz_ru, 
                  key_on='feature.properties.TAZ_ID',
                  columns=['TAZ_ID', 'LU_Type'], 
                  fill_color='YlGnBu').add_to(m)

folium.LayerControl().add_to(m)

m

## Exports for Testing

In [ ]:
outfp = r"C:\Users\matkinson\Downloads\taz_ru_25.shp"
taz_ru.to_file(outfp)